In [1]:
import pandas as pd
import numpy as np
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
'done'

'done'

In [2]:
df = pd.concat([train,test], axis = 0, sort = False)

In [3]:
# One hot

In [4]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object'] + ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

factor_columns = [col for col in categorical_columns if len(set(df[col])) == 2] + ['CODE_GENDER', 'WEEKDAY_APPR_PROCESS_START']

ori_cat = [i for i in categorical_columns if i not in factor_columns]

# Factorize

In [5]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'] + categorical_columns:
    print(bin_feature)
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [6]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

# AREA

In [7]:
print(df.shape)
l = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
       'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
       'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
       'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
       'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
       'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
       'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
       'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
       'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE',
       'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

(356255, 122)


In [8]:
x = df[l]
x.fillna(0 , inplace = True)
df[l] = x

/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
area = ['APARTMENTS','BASEMENTAREA', 'COMMONAREA','LANDAREA', 'LIVINGAPARTMENTS','LIVINGAREA',\
           'NONLIVINGAPARTMENTS', 'NONLIVINGAREA']

for x in ['_AVG', '_MODE', '_MEDI']:
    tmp = [y + x for y in area]
    print(tmp)
    df['livingsum' + x] = df[tmp].sum(axis=1)

['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'COMMONAREA_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']
['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE']
['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI']


In [10]:
df.shape

(356255, 125)

In [11]:
df[['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'COMMONAREA_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']]

,APARTMENTS_AVG,BASEMENTAREA_AVG,COMMONAREA_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG
0,0.0247,0.0369,0.0143,0.0369,0.0202,0.0190,0.0000,0.0000
1,0.0959,0.0529,0.0605,0.0130,0.0773,0.0549,0.0039,0.0098
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


# 1. Do the ratio wrt to count of family members & living area

In [12]:
# 1. Ratio of count of family members(for members less than 4) and living area

In [13]:
members_series = df.CNT_FAM_MEMBERS < 4

area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']

In [14]:
df['members'] = np.nan

df['members'].loc[members_series] = df['CNT_FAM_MEMBERS'].loc[members_series]

for col in area_list:
    df[ratio_name(col, 'members')] = ratio(df, col, 'members')
    print(ratio_name(col, 'members'))
    
df.drop('members', inplace = True,axis = 1)

livingsum_AVG_divide_members
livingsum_MODE_divide_members
livingsum_MEDI_divide_members


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
df.shape

(356255, 128)

In [16]:
df['livingsum_AVG_divide_members'].head()

0    0.1520
1    0.1841
2    0.0000
3    0.0000
4    0.0000
Name: livingsum_AVG_divide_members, dtype: float64

# 2. Multiply area rating & living area

In [17]:
# def multiply_name(col1, col2): return col1 + '_times_' + col2
# def multiply(df, col1, col2): return df[col1] * df[col2]

In [18]:
rating = ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']
area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']
for i in rating:
    for j in area_list:
        df[multiply_name(i, j)] = multiply(df, i, i)
        print(multiply_name(i, j))

REGION_RATING_CLIENT_times_livingsum_AVG
REGION_RATING_CLIENT_times_livingsum_MODE
REGION_RATING_CLIENT_times_livingsum_MEDI
REGION_RATING_CLIENT_W_CITY_times_livingsum_AVG
REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE
REGION_RATING_CLIENT_W_CITY_times_livingsum_MEDI


In [19]:
df.shape

(356255, 134)

# Do the ratio wrt to amt-dpd and region-population

In [20]:
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
pop = ['REGION_POPULATION_RELATIVE']

In [21]:
for col in obs:
    df[ratio_name(col, 'REGION_POPULATION_RELATIVE')] = ratio(df, col, 'REGION_POPULATION_RELATIVE')
    print(ratio_name(col, 'REGION_POPULATION_RELATIVE'))

OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
OBS_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE


In [22]:
df[['OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE']]

,OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
0,106.377320
1,282.406100
2,0.000000
3,249.407657
4,0.000000
5,0.000000
6,27.939204
7,640.614990
8,53.665343
9,101.579562


In [23]:
df.shape

(356255, 138)

# DEF/OBS & 60/30

In [24]:
obs_30 = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE']
obs_60 = ['OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']

In [25]:
for i in obs_60:
    for j in obs_30:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)

OBS_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE


In [26]:
df.shape

(356255, 142)

In [27]:
default = ['DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']

In [28]:
for i in default:
    for j in obs:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)
        print(df.shape)

DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 143)
DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 144)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 144)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 145)


# Combine phone columns, and count how many flags

In [29]:
df[['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE']].isnull().sum()

FLAG_MOBIL          0
FLAG_EMP_PHONE      0
FLAG_WORK_PHONE     0
FLAG_CONT_MOBILE    0
FLAG_PHONE          0
dtype: int64

In [30]:
df['CONTACT'] = df[['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE','FLAG_EMAIL']].sum(axis = 1)

df.drop(['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE'], axis = 1, inplace = True)

In [31]:
df['FLAG_OTHER'] = df[['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY']].sum(axis = 1)

In [32]:
set(df['FLAG_OTHER'].values)

{0, 1, 2}

In [33]:
df.shape

(356255, 142)

In [34]:
# x = df.copy()
# x[['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
#        'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
#        'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']].dropna(inplace = True)

In [35]:
# Train weighted sum of inq
df['req_total'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'] +\
                        df['AMT_REQ_CREDIT_BUREAU_DAY']*0.9+\
                        df['AMT_REQ_CREDIT_BUREAU_WEEK']*0.8+\
                        df['AMT_REQ_CREDIT_BUREAU_MON']*0.7+\
                        df['AMT_REQ_CREDIT_BUREAU_QRT']*0.6+\
                        df['AMT_REQ_CREDIT_BUREAU_YEAR']*0.5

In [36]:
df.shape

(356255, 143)

# Other Ratios:

In [37]:
can_be_numerator = [x for x in train.columns if 'AMT' in x] + ['OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
can_be_denominator = [x for x in train.columns if 'CNT' in x or 'DAYS' in x] + can_be_numerator

print(len(can_be_numerator),can_be_numerator)
print(len(can_be_denominator),can_be_denominator)

12 ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
23 ['CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']


In [38]:
i = 0
for x in can_be_numerator:
    for y in can_be_denominator:
        if y != x and ratio_name(y,x) not in df.columns:
            i += 1
            print(ratio_name(x,y))
            df[ratio_name(x, y)] = ratio(df, x, y)
print(i, df.shape)

AMT_INCOME_TOTAL_divide_CNT_CHILDREN
AMT_INCOME_TOTAL_divide_DAYS_BIRTH
AMT_INCOME_TOTAL_divide_DAYS_EMPLOYED
AMT_INCOME_TOTAL_divide_DAYS_REGISTRATION
AMT_INCOME_TOTAL_divide_DAYS_ID_PUBLISH
AMT_INCOME_TOTAL_divide_CNT_FAM_MEMBERS
AMT_INCOME_TOTAL_divide_OBS_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_OBS_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_INCOME_TOTAL_divide_AMT_CREDIT
AMT_INCOME_TOTAL_divide_AMT_ANNUITY
AMT_INCOME_TOTAL_divide_AMT_GOODS_PRICE
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_HOUR
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_DAY
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_WEEK
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_MON
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_QRT
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_YEAR
AMT_INCOME_TOTAL_divide_OWN_CAR_AGE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_CREDIT_divide_CNT_CHILDR

DAYS_LAST_PHONE_CHANGE_divide_OBS_60_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_DEF_60_CNT_SOCIAL_CIRCLE
197 (356255, 329)


In [39]:
# One hot
print(df.shape)
ignore_cat = ['ORGANIZATION_TYPE'] # 'NAME_HOUSING_TYPE','OCCUPATION_TYPE',
for col in ori_cat:
    if col not in ignore_cat:
        tmp = pd.get_dummies(df[col], prefix= 'one_hot'+col, dummy_na = True)
        print(tmp.shape)
        print(col)
        df = pd.concat([df, tmp], axis = 1)
        print(df.shape)
#     print(tmp[:5])
print(df.shape)

df.drop(ori_cat, axis = 1, inplace = True)
print(df.shape)

(356255, 329)
(356255, 9)
NAME_TYPE_SUITE
(356255, 338)
(356255, 9)
NAME_INCOME_TYPE
(356255, 347)
(356255, 6)
NAME_EDUCATION_TYPE
(356255, 353)
(356255, 7)
NAME_FAMILY_STATUS
(356255, 360)
(356255, 7)
NAME_HOUSING_TYPE
(356255, 367)
(356255, 20)
OCCUPATION_TYPE
(356255, 387)
(356255, 6)
FONDKAPREMONT_MODE
(356255, 393)
(356255, 5)
HOUSETYPE_MODE
(356255, 398)
(356255, 9)
WALLSMATERIAL_MODE
(356255, 407)
(356255, 4)
EMERGENCYSTATE_MODE
(356255, 411)
(356255, 411)
(356255, 400)


# Cat Features

In [40]:


train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
print(train.shape,test.shape)

test.drop('TARGET', axis = 1, inplace = True)
test.shape

(307511, 400) (48744, 400)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(48744, 399)

In [41]:
train.to_pickle(PATH + 'test/train_only_2.pkl')
test.to_pickle(PATH + 'test/test_only_2.pkl')

In [42]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,one_hotWALLSMATERIAL_MODE_2.0,one_hotWALLSMATERIAL_MODE_3.0,one_hotWALLSMATERIAL_MODE_4.0,one_hotWALLSMATERIAL_MODE_5.0,one_hotWALLSMATERIAL_MODE_6.0,one_hotWALLSMATERIAL_MODE_nan,one_hotEMERGENCYSTATE_MODE_-1.0,one_hotEMERGENCYSTATE_MODE_0.0,one_hotEMERGENCYSTATE_MODE_1.0,one_hotEMERGENCYSTATE_MODE_nan
0,100002,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0,0,0,0,0,1,0,0
1,100003,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,0,0,0,1,0,0,0
3,100006,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,0,1,0,0,0
4,100007,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,0,0,0,1,0,0,0
